In [8]:
# Author : Ayush

# Downloading the dataset
from sklearn.datasets import fetch_20newsgroups
twenty_train = fetch_20newsgroups(subset='train', shuffle=True)

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import preprocessing
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction import text
from nltk.stem.snowball import EnglishStemmer
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer
import numpy as np
import pickle
#from sklearn.externals import joblib

class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, articles):
        return [self.wnl.lemmatize(t) for t in word_tokenize(articles)]

stemmer = EnglishStemmer(ignore_stopwords=True)
analyzer = CountVectorizer().build_analyzer()
custom_stop_words=['english', 'slack', 'thanks', 'welcome']
stop_words = text.ENGLISH_STOP_WORDS.union(custom_stop_words)
text_clf = Pipeline([('vect', CountVectorizer(stop_words=stop_words,tokenizer=LemmaTokenizer(),lowercase = True, token_pattern=r'\b[^\d\W]+\b')),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB())
                    ])

text_clf = text_clf.fit(twenty_train.data, twenty_train.target)
twenty_test = fetch_20newsgroups(subset='test', shuffle=True)
predicted = text_clf.predict(twenty_test.data)
print("Accuracy = "+str(np.mean(predicted == twenty_test.target)))

# save the model to disk
#filename = 'finalized_model.sav'
#joblib.dump(text_clf, filename)

pkl_filename = "pipeline_model.pkl"  
with open(pkl_filename, 'wb') as file:  
    pickle.dump(text_clf, file)

/usr/local/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


Accuracy = 0.8019118428040362


In [7]:
# This is for testing out stemming and CountVectorizer
from sklearn.feature_extraction import text
from nltk.stem.snowball import EnglishStemmer
import pandas as pd

stemmer = EnglishStemmer()
analyzer = CountVectorizer().build_analyzer()

def stemmed_words(doc):
    return (stemmer.stem(w) for w in analyzer(doc))

df = pd.DataFrame(['my the the the in 123abc an the bigger dog win grinning stunning', 'my 345 this be lazy cat winning'])
value_list = [row[0] for row in df.itertuples(index=False, name=None)]
my_additional_stop_words=['my', 'big', 'english']
stop_words = text.ENGLISH_STOP_WORDS.union(my_additional_stop_words)

cv = CountVectorizer(stop_words=stop_words, token_pattern=r'\b[^\d\W]+\b', analyzer=stemmed_words)

x_train = cv.fit_transform(value_list)
print(x_train.toarray())
print(cv.vocabulary_)

[[1 0 1 0 1 0 1 1 1 0 1 1 4 0 1]
 [0 1 0 1 0 1 0 0 0 1 1 0 0 1 1]]
{'my': 10, 'the': 12, 'in': 8, '123abc': 0, 'an': 2, 'bigger': 4, 'dog': 6, 'win': 14, 'grin': 7, 'stun': 11, '345': 1, 'this': 13, 'be': 3, 'lazi': 9, 'cat': 5}
